Given a Run number, the file with the s2 and s2si data of interest for that Run (i.e Th photopeak, Cs,..), this nb plots the trace for a given event.

In [1]:
import os

import invisible_cities.io.pmaps_io as pm
from invisible_cities.database.load_db import DataSiPM

import pandas as pd
import numpy as np

import matplotlib as mpl
mpl.use('Qt5Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.widgets import Slider

## useful functions

In [2]:
def _each(edges, n):
    '''Given a list, it given 2 lists such as the first is the original with values
    in positions 0,0+n, 0+2*n ...'''
    
    if len(edges)%n:
        mat = []
        mit = []
        i = 0
        while i<len(edges)-n:
            mat.append(edges[i])
            for j in range(1,n): mit.append(edges[i+j])
            i += n
        mat.append(edges[-1])
        return mat, mit
    else: raise ValueError("number of edges do not match with n")

# Sipm positions

In [3]:
run = 6206
datasipm = DataSiPM(run)

# EVENT CHOICE

In [4]:
evt_type = 'descape' #string with event type: photopeakTh, cs, descape

evts = np.load("/home/gonzalo/Documentos/NEXT/IMAGES/data selection/{}_evts_{}.npz".format(evt_type, run))['A_evtnum']

In [5]:
n_ev = 1000 #this value should be fixed
evts_part  = [evts[n_ev*i:(i+1)*n_ev] for i in range(0, int(len(evts)/n_ev))] + [evts[n_ev * int(len(evts)/n_ev):]]

In [6]:
#choose an event
j = int(np.random.random()*(int(len(evts)/n_ev)-1))
i = int(np.random.random()*n_ev)-1

ev = evts_part[j][i]

In [7]:
#data of selected events
data  = pd.HDFStore('/home/gonzalo/Documentos/NEXT/IMAGES/data selection/{}/{}_{}_{}.h5'.format(run,evt_type, run,j))

s1, s2, s2si, s1pmt, s2pmt = data['s1'], data['s2'], data['s2si'], data['s1pmt'], data['s2pmt']

In [8]:
s1, s2_ev, s2si_ev, s1pmt, s2pmt = s1[s1.event == ev], s2[s2.event == ev], s2si[s2si.event == ev], s1pmt[s1pmt.event == ev], s2pmt[s2pmt.event == ev]

# IMAGE DATAFRAME

In [9]:
times = np.array(s2_ev['time'])

In [10]:
#INSERTING TIMES TO SIPM DATAFRAME
df1 = s2si_ev.reset_index(drop=True)
df2_1 = s2_ev['time'].to_frame().reset_index(drop=True)
n_times = len(df2_1)

df2 = pd.concat([df2_1, df2_1], ignore_index=True)
while len(df2)<len(df1): 
    df2 = pd.concat([df2, df2_1], ignore_index=True)

df = pd.concat([df1, df2], axis = 1)

In [11]:
# INSETING SIPM POSITION TO THE NEW SIPM DATAFRAME
si_pos = pd.DataFrame(columns = ['X', 'Y'])

sipms = df1.loc[[n_times*i for i in range(0,int(len(df1)/n_times))]]['nsipm']

X, Y = pd.Series([], name='X'), pd.Series([], name='Y')

for sipm in sipms:
    
    si_info = datasipm.loc[sipm][['X','Y']]
    
    pos = np.array(si_info)*np.ones([n_times,2])
    
    X = X.append(pd.Series(pos[:,0]), ignore_index = True)
    Y = Y.append(pd.Series(pos[:,1]), ignore_index = True) 
    
si_pos['X'], si_pos['Y'] = X, Y

In [12]:
imagedf = pd.concat([df, si_pos], axis=1)

# Movie method

In [13]:
d = 10 #distancia entre sipm
x, y =  datasipm['X'], datasipm['Y']

xmin, xmax =  x.min(), x.max()
ymin, ymax =  y.min(), y.max()

nx, ny = (xmax-xmin)/10 + 1, (ymax-ymin)/10 + 1
rg = [[xmin-d/2, xmax+d/2], [ymin-d/2, ymax+d/2]]

enemin, enemax  = imagedf['ene'].min(), imagedf['ene'].max()

H, xedges, yedges = np.histogram2d(x, y, bins = [nx, ny], range = rg)
H = H.T

si_enes = []
for time in times:
    si_enes.append(imagedf[imagedf.time == time]['ene'].sum())

In [14]:
#detector path
verts = [(-80, -240), (80, -240)  , (80, -200)  , (160, -200) , (160, -120),
         (240, -120), (240, 120)  , (160, 120)  , (160, 200)  , (80, 200)  ,
         (80, 240)  , (-80, 240)  , (-80, 200)  , (-160, 200) , (-160, 120),
         (-240, 120), (-240, -120), (-160, -120), (-160, -200), (-80, -200),
         (-80, -240)]

codes = [ Path.MOVETO, Path.LINETO, Path.LINETO, Path.LINETO, Path.LINETO,
          Path.LINETO, Path.LINETO, Path.LINETO, Path.LINETO, Path.LINETO,
          Path.LINETO, Path.LINETO, Path.LINETO, Path.LINETO, Path.LINETO,
          Path.LINETO, Path.LINETO, Path.LINETO, Path.LINETO, Path.LINETO,
          Path.CLOSEPOLY,]

path  = Path(verts, codes)
patch = patches.PathPatch(path, facecolor='none', edgecolor='black', lw=3)

In [17]:
fig = plt.figure(figsize=(10, 5))

spec = gridspec.GridSpec(ncols=2, nrows=1, left=0.07, right=0.93, wspace=0.2)

ax  = fig.add_subplot(spec[0, 0], title='NEXT-100 tracking plane')
ax1 = fig.add_subplot(spec[0, 1], title='E vs T')

fig.text( .8, .95  , 'Run number: {}'.format(run));
fig.text( .8, .9  , 'Event number: {}'.format(ev));
fig.text( .8, .85  , 'Event type: {}'.format(evt_type));
e = fig.text( .8, .75 , 'Energy: {}'   .format(''));
t = fig.text( .8, .70 , 'Time: {}'     .format(''));

# TRACKING PLANE AXES
#ax  = fig.add_subplot(1,2,1, title='NEXT-100 tracking plane')
axim = ax.imshow(H, cmap='jet', vmin = enemin , vmax = enemax  ,interpolation='nearest', origin='low',
           extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], alpha = 0.5)
#ticks of 1 cm^2
maxt, mixt = _each(xedges, n = 4)
ax.set_xticks(maxt, minor = False)
ax.set_xticks(mixt, minor = True)

mayt, miyt = _each(yedges, n = 2)
ax.set_yticks(mayt, minor = False)
ax.set_yticks(miyt, minor = True)

# grid,detector contourn
ax.grid(which='both');
ax.patch.set_animated(True);
ax.add_patch(patch);

#colorbar
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.07)
cbar = fig.colorbar(axim, cax = cax);

# E vs T AXES
#ax1   = fig.add_subplot(1,2,2, title='E vs T')
line, = ax1.plot(times, si_enes)

#draw circle (taken from https://werthmuller.org/blog/2014/circle/)
xy     = (times[0], si_enes[0]) # circle centre
radius = .01

pr      = fig.get_figwidth()/fig.get_figheight()
tscale  = ax1.transScale + (ax1.transLimits + ax1.transAxes)
ctscale = tscale.transform_point(xy)
cfig    = fig.transFigure.inverted().transform(ctscale)

cir = patches.Ellipse(cfig, radius, radius*pr,
                transform=fig.transFigure, facecolor = 'none', edgecolor = 'r')

ax1.add_artist(cir);

In [18]:
def update(frame_number):
    
    frame_number = int(frame_number)
    
    #sl.set_val(frame_number)
    
    #tracking image
    imdf = imagedf[imagedf.time == times[frame_number]]
    x, y, w = imdf['X'], imdf['Y'], imdf['ene']
    H, __, __ = np.histogram2d(x, y, weights = w, bins = [nx, ny], range = rg)
    H = H.T
    axim.set_data(H)
    
    #circle 
    xy = (times[frame_number], w.sum())
    ctscale = tscale.transform_point(xy)
    cfig    = fig.transFigure.inverted().transform(ctscale)
    cir.center = cfig
    
    # figure text actualization
    e.set_text('Energy: {}'.format(int(w.sum())))
    t.set_text('Time: {}'  .format(int(times[frame_number])))
    
    fig.canvas.draw_idle()
    
    return axim, e, t, cir

In [19]:
#caxsl = divider.append_axes("bottom", size="5%", pad=0.3)
caxsl = plt.axes([0.1, 0.01, 0.8, 0.03], facecolor='b')

sl = Slider(caxsl, 'T', 0, len(times)-1, valinit = 0, valstep = 1)

sl.on_changed(update);

In [20]:
fig.show()